In [2]:
import setup
import qiskit
import matplotlib
import matplotlib.pyplot as plt
from circuit import QCircuit
from algo.func_optimizer import BfgsOptimizer, CmaesOptimizer
from algo.one_plus_lambda import OnePlusLambda, EvolutionReport, IterationReport
from iohelper import hamiltonians, results as rio
from circuit import QCircuit, GateTypes, QCircuitConversions
from algo.vqe import PyVqe
from mutations import Weighted, Insert, Remove, add_two_block_layers
from algo.cvqe_wrapper import CVqe

ImportError: libboost_python3.so.1.66.0: cannot open shared object file: No such file or directory

In [ ]:
def run(task, block_type, max_cmaes_iter=None, verbose=True):
    # Show information about task
    if verbose:
        print('Minimal eigenvalue is  {}'.format(task.min_eigenvalue))
        print('Initial state is {}'.format(task.classical_psi0_bitstring))

    # Initialize circuit 
    circuit = QCircuit(task.N, task.classical_psi0, [])
    add_two_block_layers(circuit, block_type)


    def on_iteration_end(report: IterationReport):
        if verbose:
            num_evaluations = sum(m.num_circ_evaluations for m in report.mutations)
            print('Iteration #{} is finished, {} circuit evaluations performed'.format(report.index, num_evaluations))
            print('  Values: {}!'.format(', '.join(str(m.value) for m in report.mutations)))
            if report.better:
                best_mutation = report.mutations[0]
                print('  New value: {}!'.format(best_mutation.value))
                circ = best_mutation.circ
                display(QCircuitConversions.to_qiskit_circuit(circ).draw(output='mpl'))


    # Run evolutional algorithm
    ev = OnePlusLambda(
        target=task.min_eigenvalue,
        vqe=CVqe(task.H, 1e-5),
        mutation=Weighted([
            (Insert(block_type), 2),
            (Remove(), 1)
        ]),
        initial=circuit,
        target_eps=1.6e-3,
        alambda=128
    )
    report: EvolutionReport = ev.run(iteration_end_callback=on_iteration_end)

    # Show results
    if report.interrupted:
        print('INTERRUPTED')

    print('Number of CNOTs: ' + str(report.best_circuit.size))
    print('Best value is: ' + str(report.best_circuit_value))
    print('Number of iterations is: ' + str(len(report.iterations) - 1))
    if verbose:
        display(QCircuitConversions.to_qiskit_circuit(report.best_circuit).draw(output='mpl'))
    return report

task = hamiltonians.q8
reports = [run(task, GateTypes.block_a, verbose=True) for i in range(0, 1)]

In [7]:
rio.store('collections', 'LiH-6', reports)

In [2]:
from qutip import expect

task = hamiltonians.q4
circuit = QCircuit(task.N, task.classical_psi0, [])
add_two_block_layers(circuit, GateTypes.block_a)
v = CVqe(task.H, 1e-14)
opt = v.optimize(circuit)
c = opt.circ
print(c.get_parameters())
print('{} - {}'.format(min(c.get_parameters()), max(c.get_parameters())))
w = QCircuitConversions.to_qobj_wavefunction(c)
print(expect(task.H, w))
print(opt.opt_value)

[-3.14159239e+00 -2.84144427e+00 -3.14159171e+00  3.28328896e-01
  3.14159161e+00 -2.83112019e-01 -3.14158870e+00  3.71486307e-01
 -3.13375862e+00 -3.14159265e+00 -3.46660561e-07  1.43243319e+00]
-3.141592653589746 - 3.141591605444185
-0.8746828757722297
-0.8746828757722289
